 # 03_fact_diario.ipynb — PROYECTO AURORA



 Construcción del FACT diario: Canal A (Steam) vs Canal B (plataforma B)

In [28]:
import pandas as pd
import numpy as np
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 120)

CLEAN_PATH = "../data/clean/"
FINAL_PATH = "../data/final/"

if not os.path.exists(FINAL_PATH):
    os.makedirs(FINAL_PATH)

# Rango de fechas válido para el análisis comparativo
FECHA_MIN = pd.Timestamp("2022-06-01")
FECHA_MAX = pd.Timestamp("2025-10-31")

print("Rango FACT:", FECHA_MIN, "→", FECHA_MAX)
print("CLEAN_PATH →", CLEAN_PATH)
print("FINAL_PATH →", FINAL_PATH)


Rango FACT: 2022-06-01 00:00:00 → 2025-10-31 00:00:00
CLEAN_PATH → ../data/clean/
FINAL_PATH → ../data/final/


In [29]:
# Mapas para nombres de mes y día en castellano
mes_map = {
    1: "enero", 2: "febrero", 3: "marzo", 4: "abril",
    5: "mayo", 6: "junio", 7: "julio", 8: "agosto",
    9: "septiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
}

dia_map = {
    0: "lunes", 1: "martes", 2: "miércoles", 3: "jueves",
    4: "viernes", 5: "sábado", 6: "domingo"
}


In [30]:
steam_dl = pd.read_csv(os.path.join(CLEAN_PATH, "steam_descargas_clean.csv"))
steam_stats = pd.read_csv(os.path.join(CLEAN_PATH, "steam_stats_clean.csv"))
steam_sales = pd.read_csv(os.path.join(CLEAN_PATH, "steam_ventas_clean.csv"))
canalB = pd.read_csv(os.path.join(CLEAN_PATH, "canalB_clean.csv"))

print("✅ Datasets limpios cargados")
print("steam_dl:", steam_dl.shape)
print("steam_stats:", steam_stats.shape)
print("steam_ventas:", steam_sales.shape)
print("canalB:", canalB.shape)



✅ Datasets limpios cargados
steam_dl: (1274, 3)
steam_stats: (1274, 4)
steam_ventas: (1273, 9)
canalB: (1278, 20)


In [31]:
def recortar_rango(df, col="Fecha"):
    df = df.copy()
    df[col] = pd.to_datetime(df[col], errors="coerce")
    mask = (df[col] >= FECHA_MIN) & (df[col] <= FECHA_MAX)
    return df[mask].copy()


steam_dl = recortar_rango(steam_dl)
steam_stats = recortar_rango(steam_stats)
steam_sales = recortar_rango(steam_sales)
canalB = recortar_rango(canalB)

print("Rangos tras recorte:")
print("Steam Descargas:", steam_dl["Fecha"].min(), "→", steam_dl["Fecha"].max())
print("Steam Stats:", steam_stats["Fecha"].min(), "→", steam_stats["Fecha"].max())
print("Steam Ventas:", steam_sales["Fecha"].min(), "→", steam_sales["Fecha"].max())
print("Canal B:", canalB["Fecha"].min(), "→", canalB["Fecha"].max())



Rangos tras recorte:
Steam Descargas: 2022-06-06 00:00:00 → 2025-10-31 00:00:00
Steam Stats: 2022-06-01 00:00:00 → 2025-10-31 00:00:00
Steam Ventas: 2022-06-01 00:00:00 → 2025-10-31 00:00:00
Canal B: 2022-06-01 00:00:00 → 2025-10-31 00:00:00


In [32]:
# Partimos de steam_stats (DAU + concurrentes)
steam_a = (
    steam_stats
    .merge(steam_sales, on="Fecha", how="left")
    .merge(steam_dl, on="Fecha", how="left")
)

print("Columnas steam_a después del merge:")
print(steam_a.columns.tolist())
steam_a.head()



Columnas steam_a después del merge:
['Fecha', 'DAU', 'Usuarios_Concurrentes_Pico', 'Juego_x', 'Unidades_Vendidas_Brutas', 'Devoluciones_Unidades', 'Unidades_Vendidas', 'Precio_Medio_USD', 'Ingresos_Brutos_USD', 'Devoluciones_USD', 'Impuestos_USD', 'Ingresos_USD', 'Descargas', 'Juego_y']


,Fecha,DAU,Usuarios_Concurrentes_Pico,Juego_x,Unidades_Vendidas_Brutas,Devoluciones_Unidades,Unidades_Vendidas,Precio_Medio_USD,Ingresos_Brutos_USD,Devoluciones_USD,Impuestos_USD,Ingresos_USD,Descargas,Juego_y
0,2022-06-01,1294,83,Proyecto Aurora,44,0,44,8.913636,449.68,0.0,-21.24,428.44,NaN,NaN
1,2022-06-02,1315,80,Proyecto Aurora,67,0,67,20.268750,1294.24,0.0,-98.28,1195.96,NaN,NaN
2,2022-06-03,1355,87,Proyecto Aurora,117,0,117,21.067500,2304.43,0.0,-174.96,2129.47,NaN,NaN
3,2022-06-04,1307,86,Proyecto Aurora,83,0,83,15.625000,1031.35,0.0,-89.98,941.37,NaN,NaN
4,2022-06-05,1378,96,Proyecto Aurora,136,0,136,12.862727,1732.55,0.0,-140.89,1591.66,NaN,NaN


In [33]:
steam_a["Plataforma"] = "Canal A"
steam_a["Juego"] = "Proyecto Aurora"

steam_a["Año"] = steam_a["Fecha"].dt.year
steam_a["Mes"] = steam_a["Fecha"].dt.month
steam_a["Trimestre"] = steam_a["Fecha"].dt.to_period("Q").astype(str)
steam_a["DiaSemana"] = steam_a["Fecha"].dt.weekday
steam_a["Mes_Nombre"] = steam_a["Mes"].map(mes_map)
steam_a["DiaSemana_Nombre"] = steam_a["DiaSemana"].map(dia_map)

steam_a.head()



,Fecha,DAU,Usuarios_Concurrentes_Pico,Juego_x,Unidades_Vendidas_Brutas,Devoluciones_Unidades,Unidades_Vendidas,Precio_Medio_USD,Ingresos_Brutos_USD,Devoluciones_USD,Impuestos_USD,Ingresos_USD,Descargas,Juego_y,Plataforma,Juego,Año,Mes,Trimestre,DiaSemana,Mes_Nombre,DiaSemana_Nombre
0,2022-06-01,1294,83,Proyecto Aurora,44,0,44,8.913636,449.68,0.0,-21.24,428.44,NaN,NaN,Canal A,Proyecto Aurora,2022,6,2022Q2,2,junio,miércoles
1,2022-06-02,1315,80,Proyecto Aurora,67,0,67,20.268750,1294.24,0.0,-98.28,1195.96,NaN,NaN,Canal A,Proyecto Aurora,2022,6,2022Q2,3,junio,jueves
2,2022-06-03,1355,87,Proyecto Aurora,117,0,117,21.067500,2304.43,0.0,-174.96,2129.47,NaN,NaN,Canal A,Proyecto Aurora,2022,6,2022Q2,4,junio,viernes
3,2022-06-04,1307,86,Proyecto Aurora,83,0,83,15.625000,1031.35,0.0,-89.98,941.37,NaN,NaN,Canal A,Proyecto Aurora,2022,6,2022Q2,5,junio,sábado
4,2022-06-05,1378,96,Proyecto Aurora,136,0,136,12.862727,1732.55,0.0,-140.89,1591.66,NaN,NaN,Canal A,Proyecto Aurora,2022,6,2022Q2,6,junio,domingo


In [34]:
cols_cero = ["Descargas", "Ingresos_USD", "Ingresos_Brutos_USD", "Unidades_Vendidas"]

for c in cols_cero:
    if c in steam_a.columns:
        steam_a[c] = steam_a[c].fillna(0)

steam_a.head()



,Fecha,DAU,Usuarios_Concurrentes_Pico,Juego_x,Unidades_Vendidas_Brutas,Devoluciones_Unidades,Unidades_Vendidas,Precio_Medio_USD,Ingresos_Brutos_USD,Devoluciones_USD,Impuestos_USD,Ingresos_USD,Descargas,Juego_y,Plataforma,Juego,Año,Mes,Trimestre,DiaSemana,Mes_Nombre,DiaSemana_Nombre
0,2022-06-01,1294,83,Proyecto Aurora,44,0,44,8.913636,449.68,0.0,-21.24,428.44,0.0,NaN,Canal A,Proyecto Aurora,2022,6,2022Q2,2,junio,miércoles
1,2022-06-02,1315,80,Proyecto Aurora,67,0,67,20.268750,1294.24,0.0,-98.28,1195.96,0.0,NaN,Canal A,Proyecto Aurora,2022,6,2022Q2,3,junio,jueves
2,2022-06-03,1355,87,Proyecto Aurora,117,0,117,21.067500,2304.43,0.0,-174.96,2129.47,0.0,NaN,Canal A,Proyecto Aurora,2022,6,2022Q2,4,junio,viernes
3,2022-06-04,1307,86,Proyecto Aurora,83,0,83,15.625000,1031.35,0.0,-89.98,941.37,0.0,NaN,Canal A,Proyecto Aurora,2022,6,2022Q2,5,junio,sábado
4,2022-06-05,1378,96,Proyecto Aurora,136,0,136,12.862727,1732.55,0.0,-140.89,1591.66,0.0,NaN,Canal A,Proyecto Aurora,2022,6,2022Q2,6,junio,domingo


In [35]:
columnas_fact = [
    "Fecha",
    "Año", "Trimestre", "Mes", "Mes_Nombre",
    "DiaSemana", "DiaSemana_Nombre",
    "Plataforma", "Juego",
    "DAU",
    "Descargas",
    "Ingresos_USD",
    "Ingresos_Brutos_USD",
    "Unidades_Vendidas",
    "Usuarios_Concurrentes_Pico",
    "Oro_Gastado_USD",
    "ARPDAU",
    "ARPU",
    "ARPPU",
]

# Columnas que solo existen en Canal B -> NaN en Canal A
steam_a["Oro_Gastado_USD"] = np.nan
steam_a["ARPDAU"] = np.nan
steam_a["ARPU"] = np.nan
steam_a["ARPPU"] = np.nan

# Airbag por si falta alguna columna del esquema
for col in columnas_fact:
    if col not in steam_a.columns:
        steam_a[col] = np.nan

steam_a_fact = steam_a[columnas_fact].copy()
steam_a_fact.head()



,Fecha,Año,Trimestre,Mes,Mes_Nombre,DiaSemana,DiaSemana_Nombre,Plataforma,Juego,DAU,Descargas,Ingresos_USD,Ingresos_Brutos_USD,Unidades_Vendidas,Usuarios_Concurrentes_Pico,Oro_Gastado_USD,ARPDAU,ARPU,ARPPU
0,2022-06-01,2022,2022Q2,6,junio,2,miércoles,Canal A,Proyecto Aurora,1294,0.0,428.44,449.68,44,83,NaN,NaN,NaN,NaN
1,2022-06-02,2022,2022Q2,6,junio,3,jueves,Canal A,Proyecto Aurora,1315,0.0,1195.96,1294.24,67,80,NaN,NaN,NaN,NaN
2,2022-06-03,2022,2022Q2,6,junio,4,viernes,Canal A,Proyecto Aurora,1355,0.0,2129.47,2304.43,117,87,NaN,NaN,NaN,NaN
3,2022-06-04,2022,2022Q2,6,junio,5,sábado,Canal A,Proyecto Aurora,1307,0.0,941.37,1031.35,83,86,NaN,NaN,NaN,NaN
4,2022-06-05,2022,2022Q2,6,junio,6,domingo,Canal A,Proyecto Aurora,1378,0.0,1591.66,1732.55,136,96,NaN,NaN,NaN,NaN


In [36]:
canalB_b = canalB.copy()

canalB_b["Plataforma"] = "Canal B"
canalB_b["Juego"] = "Proyecto Aurora"

canalB_b["Año"] = canalB_b["Fecha"].dt.year
canalB_b["Mes"] = canalB_b["Fecha"].dt.month
canalB_b["Trimestre"] = canalB_b["Fecha"].dt.to_period("Q").astype(str)
canalB_b["DiaSemana"] = canalB_b["Fecha"].dt.weekday
canalB_b["Mes_Nombre"] = canalB_b["Mes"].map(mes_map)
canalB_b["DiaSemana_Nombre"] = canalB_b["DiaSemana"].map(dia_map)

canalB_b.head()



,Fecha,APP ID,Game Name,Device,Total Registered Members,Total Verified Members,Newly Registered Members,Newly Verified Members,Newly Unregistered Members,Descargas,DAU,ARPDAU,Charged UU,Oro_Gastado,ARPU,ARPPU,Complimentary Gold Spent,Ingresos_USD,Oro_Gastado_USD,Juego,Plataforma,Año,Mes,Trimestre,DiaSemana,Mes_Nombre,DiaSemana_Nombre
0,2022-06-01,50784,Eros Fantasy,TOTAL,211492,87826,255,107,0,42,5020,5.47,9,27437,0.13,3048.56,0,274.37,274.37,Proyecto Aurora,Canal B,2022,6,2022Q2,2,junio,miércoles
1,2022-06-02,50784,Eros Fantasy,TOTAL,211862,87956,370,130,0,52,5147,9.08,21,46731,0.22,2225.29,0,467.31,467.31,Proyecto Aurora,Canal B,2022,6,2022Q2,3,junio,jueves
2,2022-06-03,50784,Eros Fantasy,TOTAL,212369,88140,507,184,0,84,5170,24.29,30,125566,0.59,4185.53,0,1255.66,1255.66,Proyecto Aurora,Canal B,2022,6,2022Q2,4,junio,viernes
3,2022-06-04,50784,Eros Fantasy,TOTAL,212729,88276,361,136,1,60,4875,12.15,27,59213,0.28,2193.07,0,592.13,592.13,Proyecto Aurora,Canal B,2022,6,2022Q2,5,junio,sábado
4,2022-06-05,50784,Eros Fantasy,TOTAL,213111,88435,382,159,0,66,4932,17.01,28,83913,0.39,2996.89,0,839.13,839.13,Proyecto Aurora,Canal B,2022,6,2022Q2,6,junio,domingo


In [37]:
# En Canal B ya tenemos: DAU, Descargas, Oro_Gastado, ARPDAU, ARPU, ARPPU, Ingresos_USD
# Creamos columnas de Steam como NaN
if "Ingresos_Brutos_USD" not in canalB_b.columns:
    canalB_b["Ingresos_Brutos_USD"] = np.nan
if "Unidades_Vendidas" not in canalB_b.columns:
    canalB_b["Unidades_Vendidas"] = np.nan
if "Usuarios_Concurrentes_Pico" not in canalB_b.columns:
    canalB_b["Usuarios_Concurrentes_Pico"] = np.nan

# Airbag: aseguramos todas las columnas del esquema
for col in columnas_fact:
    if col not in canalB_b.columns:
        canalB_b[col] = np.nan

canalB_fact = canalB_b[columnas_fact].copy()
canalB_fact.head()



,Fecha,Año,Trimestre,Mes,Mes_Nombre,DiaSemana,DiaSemana_Nombre,Plataforma,Juego,DAU,Descargas,Ingresos_USD,Ingresos_Brutos_USD,Unidades_Vendidas,Usuarios_Concurrentes_Pico,Oro_Gastado_USD,ARPDAU,ARPU,ARPPU
0,2022-06-01,2022,2022Q2,6,junio,2,miércoles,Canal B,Proyecto Aurora,5020,42,274.37,NaN,NaN,NaN,274.37,5.47,0.13,3048.56
1,2022-06-02,2022,2022Q2,6,junio,3,jueves,Canal B,Proyecto Aurora,5147,52,467.31,NaN,NaN,NaN,467.31,9.08,0.22,2225.29
2,2022-06-03,2022,2022Q2,6,junio,4,viernes,Canal B,Proyecto Aurora,5170,84,1255.66,NaN,NaN,NaN,1255.66,24.29,0.59,4185.53
3,2022-06-04,2022,2022Q2,6,junio,5,sábado,Canal B,Proyecto Aurora,4875,60,592.13,NaN,NaN,NaN,592.13,12.15,0.28,2193.07
4,2022-06-05,2022,2022Q2,6,junio,6,domingo,Canal B,Proyecto Aurora,4932,66,839.13,NaN,NaN,NaN,839.13,17.01,0.39,2996.89


In [38]:
fact_diario = pd.concat([steam_a_fact, canalB_fact], ignore_index=True)

fact_diario = fact_diario.sort_values(["Fecha", "Plataforma"]).reset_index(drop=True)

print("Shape FACT:", fact_diario.shape)
fact_diario.head()



Shape FACT: (2596, 19)


,Fecha,Año,Trimestre,Mes,Mes_Nombre,DiaSemana,DiaSemana_Nombre,Plataforma,Juego,DAU,Descargas,Ingresos_USD,Ingresos_Brutos_USD,Unidades_Vendidas,Usuarios_Concurrentes_Pico,Oro_Gastado_USD,ARPDAU,ARPU,ARPPU
0,2022-06-01,2022,2022Q2,6,junio,2,miércoles,Canal A,Proyecto Aurora,1294,0.0,428.44,449.68,44.0,83.0,NaN,NaN,NaN,NaN
1,2022-06-01,2022,2022Q2,6,junio,2,miércoles,Canal B,Proyecto Aurora,5020,42.0,274.37,NaN,NaN,NaN,274.37,5.47,0.13,3048.56
2,2022-06-02,2022,2022Q2,6,junio,3,jueves,Canal A,Proyecto Aurora,1315,0.0,1195.96,1294.24,67.0,80.0,NaN,NaN,NaN,NaN
3,2022-06-02,2022,2022Q2,6,junio,3,jueves,Canal B,Proyecto Aurora,5147,52.0,467.31,NaN,NaN,NaN,467.31,9.08,0.22,2225.29
4,2022-06-03,2022,2022Q2,6,junio,4,viernes,Canal A,Proyecto Aurora,1355,0.0,2129.47,2304.43,117.0,87.0,NaN,NaN,NaN,NaN


In [39]:
print("Plataformas en FACT:", fact_diario["Plataforma"].unique())
print("Juegos en FACT:", fact_diario["Juego"].unique())
print("Rango fechas FACT:", fact_diario["Fecha"].min(), "→", fact_diario["Fecha"].max())

fact_diario.groupby("Plataforma")[["DAU", "Descargas", "Ingresos_USD"]].sum()



Plataformas en FACT: ['Canal A' 'Canal B']
Juegos en FACT: ['Proyecto Aurora']
Rango fechas FACT: 2022-06-01 00:00:00 → 2025-10-31 00:00:00


,DAU,Descargas,Ingresos_USD
Plataforma,,,
Canal A,2016447,74508.0,1256988.89
Canal B,4048675,213439.0,941352.55


In [40]:
ruta_fact = os.path.join(FINAL_PATH, "fact_diario.csv")
fact_diario.to_csv(ruta_fact, index=False)

print(f"✅ FACT DIARIO guardado en: {ruta_fact}")




✅ FACT DIARIO guardado en: ../data/final/fact_diario.csv


📌 FACT_DIARIO — PROYECTO AURORA

- Nivel de detalle: día x plataforma (Canal A vs Canal B)
- Rango: 2022-06-01 → 2025-10-31 (noviembre excluido por calidad de datos)
- Alias:
    - Canal A = Steam
    - Canal B = Plataforma B (Notaku)
    - Juego = Proyecto Aurora

- Métricas incluidas:
    - Usuarios:
        - DAU
        - Usuarios_Concurrentes_Pico (solo Canal A)
    - Adquisición:
        - Descargas
    - Monetización:
        - Ingresos_USD
        - Ingresos_Brutos_USD (solo Canal A)
        - Unidades_Vendidas (solo Canal A)
        - Oro_Gastado_USD (solo Canal B)
        - ARPDAU, ARPU, ARPPU (solo Canal B)

Este CSV es el dataset maestro para el dashboard de Tableau y el análisis comparativo.